In [ ]:
from qiskit.visualization import plot_histogram
import numpy as np
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp
from qiskit import Aer
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# 系统参数配置
n = 2  # 作业数量
m = 3  # 机器数量
l = 2  # C_max二进制位数
N = int(np.ceil(np.log2(m)))  # 机器编码位数
p = np.array([[1, 2, 3], [2, 3, 1]])  # 处理时间矩阵

# 合法机器编码表 (二进制)
valid_machines = {
    0: [0, 0],  # 机器0: 00
    1: [0, 1],  # 机器1: 01
    2: [1, 0]  # 机器2: 10
}

# 非法机器编码 (示例: 11)
illegal_machines = [[1, 1]]

# 量子比特数 = 作业分配(n*N) + C_max(l)
num_qubits = n * N + l

# 构建QUBO矩阵
Q = np.zeros((num_qubits, num_qubits))
K1 = 1000  # 非法分配惩罚系数
P = 2000  # 时间约束惩罚系数


def machine_indicator(qubit_start, machine_bits):
    """生成机器指示函数的泡利项系数"""
    terms = {}
    for k in range(N):
        q = qubit_start + k
        terms[(q, q)] = 0.5 - machine_bits[k]  # 线性项系数
        for j in range(k + 1, N):
            terms[(q, qubit_start + j)] = 0.25  # 二次项系数
    return terms


# ----------------- 1. 非法分配惩罚项 -----------------
for illegal in illegal_machines:
    for i in range(n):
        start = i * N
        terms = machine_indicator(start, illegal)
        # 将系数添加到QUBO矩阵
        for (q, r), coeff in terms.items():
            Q[q, r] += K1 * coeff

# ----------------- 2. 时间约束项 -----------------
for machine_id, bits in valid_machines.items():
    for i in range(n):
        start = i * N
        p_ij = p[i][machine_id]

        # 生成机器分配指示函数项
        terms = machine_indicator(start, bits)

        # 添加处理时间平方项
        for (q, r), coeff in terms.items():
            Q[q, r] += P * (p_ij ** 2) * coeff

        # 添加与C_max的交叉项
        for h in range(l):
            z = n * N + h
            coeff = -2 * P * p_ij * (2 ** h)
            for k in range(N):
                q = start + k
                Q[q, z] += coeff * (0.5 - bits[k])

# ----------------- 3. C_max自身项 -----------------
for h in range(l):
    z = n * N + h
    Q[z, z] += 2 ** h  # 线性项最小化C_max
    Q[z, z] += P * (2 ** (2 * h)) * n  # 平方项

# 对称化QUBO矩阵
Q = (Q + Q.T) / 2

# 转换为哈密顿量
pauli_terms = []
for i in range(num_qubits):
    for j in range(i, num_qubits):
        if Q[i, j] != 0:
            pauli_str = ['I'] * num_qubits
            pauli_str[i] = 'Z'
            if j != i:
                pauli_str[j] = 'Z'
            pauli_terms.append((''.join(pauli_str), Q[i, j]))
hamiltonian = PauliSumOp.from_list(pauli_terms)


# print(hamiltonian)

# ----------------- QAOA 执行 -----------------
class EnhancedQAOA:
    def __init__(self, hamiltonian, max_layers=3, trials_per_layer=10):
        """
        简化版QAOA优化器
        :param hamiltonian: 问题哈密顿量
        :param max_layers: 最大优化层数
        :param trials_per_layer: 每层参数尝试次数
        """
        self.hamiltonian = hamiltonian
        self.max_layers = max_layers
        self.trials_per_layer = trials_per_layer
        self.optimal_params = []
        self.optimal_energy = float('inf')
        self.backend = Aer.get_backend('qasm_simulator')

    def optimize(self):
        """执行分层优化"""
        start_time = time.time()

        for layer in range(1, self.max_layers + 1):
            print(f"Optimizing layer {layer}/{self.max_layers}")
            params_set = self._generate_params(layer)

            for i, params in enumerate(params_set):
                result = self._run_qaoa(layer, params)
                energy = result.eigenvalue.real

                if energy < self.optimal_energy:
                    self._update_optimal(result, energy, layer)
                    print(f"  New optimal: Energy={energy:.4f}, Layer={layer}")

            if self._should_stop():
                print("Early stopping triggered")
                break

        print(f"Total time: {time.time() - start_time:.2f}s")
        return self.optimal_params, self.optimal_energy

    def _generate_params(self, layer):
        """生成当前层参数集"""
        if layer == 1:
            return [np.random.uniform(0, 2 * np.pi, 2)
                    for _ in range(self.trials_per_layer)]

        # 高层参数基于历史最优扩展
        new_params = []
        for _ in range(self.trials_per_layer):
            params = self.optimal_params.copy()
            needed = 2 * layer - len(params)
            params.extend(np.random.uniform(0, 2 * np.pi, needed))
            new_params.append(params)
        return new_params

    def _run_qaoa(self, layer, params):
        """执行QAOA优化"""
        qaoa = QAOA(
            optimizer=COBYLA(maxiter=100),
            reps=layer,
            quantum_instance=self.backend,
            initial_point=list(params)  # 确保参数为列表
        )
        return qaoa.compute_minimum_eigenvalue(self.hamiltonian)

    def _update_optimal(self, result, energy, layer):
        """更新最优解"""
        self.optimal_energy = energy
        self.optimal_params = list(result.optimal_parameters.values()) \
            if isinstance(result.optimal_parameters, dict) \
            else list(result.optimal_parameters)

        # 参数维度验证
        assert len(self.optimal_params) == 2 * layer, \
            f"参数维度错误: 预期{2 * layer}, 实际{len(self.optimal_params)}"

    def _should_stop(self):
        """早停条件检查"""
        if len(self.optimal_params) < 4:
            return False
        # 简单判断参数变化率
        params_change = np.std(self.optimal_params[-4:])
        return params_change < 0.1  # 参数变化小于10%则停止


# # ================== 系统配置和哈密顿量构建 ==================
# （此处使用用户提供的原始代码构建哈密顿量）
def decode_solution(bitstring):
    """将比特串解码为调度方案"""
    solution = {}

    # 解析作业分配
    for i in range(n):
        start = i * N
        machine_bits = [int(bitstring[start + k]) for k in range(N)]
        machine_id = int(''.join(map(str, machine_bits)), 2)
        solution[f'Job{i}'] = f'M{machine_id}' if machine_id < m else '非法分配'

    # 解析C_max
    c_max = sum(int(bitstring[n * N + h]) * (2 ** h) for h in range(l))
    solution['C_max'] = c_max

    # 验证时间约束
    machine_times = {f'M{j}': 0 for j in range(m)}
    for i in range(n):
        machine = solution[f'Job{i}']
        if machine in machine_times:
            machine_times[machine] += p[i][int(machine[1:])]
    solution['Valid'] = all(t <= c_max for t in machine_times.values())

    return solution


def analyze_results(result):
    """分析QAOA结果"""
    from qiskit.visualization import plot_histogram

    # 获取测量结果
    counts = result.eigenstate

    # 筛选合法解
    valid_solutions = {}
    total_shots = sum(counts.values())

    for bitstr, count in counts.items():
        sol = decode_solution(bitstr)
        if sol['Valid'] and '非法分配' not in sol.values():
            key = tuple(sol[f'Job{i}'] for i in range(n)) + (sol['C_max'],)
            valid_solutions[key] = valid_solutions.get(key, 0) + count
    # 打印统计信息
    print(f"合法解占比: {sum(valid_solutions.values()) / total_shots:.2%}")
    print("\nTop 5最优解:")
    sorted_sols = sorted(valid_solutions.items(), key=lambda x: (-x[1], x[0][-1]))
    for (jobs, c_max), count in sorted_sols[:5]:
        print(f"方案: {jobs}, C_max={c_max}, 占比: {count / total_shots:.2%}")

    # 可视化结果
    plot_histogram(counts, figsize=(12, 6),
                   title="完整结果分布").show()
    plot_histogram(valid_solutions, figsize=(10, 5),
                   title="合法解分布").show()


# 初始化增强型QAOA优化器
# qaoa_optimizer = EnhancedQAOA(
#     hamiltonian=hamiltonian,
#     max_layers=3,
#     trials_per_layer=10
# )

# 执行优化
# optimal_params, min_energy = qaoa_optimizer.optimize()

# 结果分析
# print("\n=== 优化结果 ===")
# print(f"最低能量值: {min_energy:.4f}")
# print("最优参数:")
# for i, (beta, gamma) in enumerate(zip(optimal_params[::2], optimal_params[1::2])):
#     print(f"  层 {i+1}: β={beta:.3f}, γ={gamma:.3f}")

# 可视化优化历史
# print("\n优化历史:")
# for record in qaoa_optimizer.history:
#     print(f"层 {record['layer']}: 能量 {record['energy']:.4f}")
optimizer = COBYLA(maxiter=1000)
quantum_instance = Aer.get_backend('qasm_simulator', shots=10000)

qaoa = QAOA(
    optimizer=optimizer,
    reps=8,
    quantum_instance=quantum_instance,
    initial_point=[1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5]  # 初始参数
)

result = qaoa.compute_minimum_eigenvalue(hamiltonian)
analyze_results(result)
# ----------------- 结果解析 -----------------
counts = result.eigenstate
print(counts)
# 可视化结果分布
plot_histogram(counts, figsize=(10, 6),
               title="QAOA结果概率分布").show()


def decode_solution(bitstring):
    """将比特字符串解码为调度方案"""
    solution = {}
    # 解析作业分配
    for i in range(n):
        start = i * N
        machine_bits = [int(bitstring[start + k]) for k in range(N)]
        machine_id = int(''.join(map(str, machine_bits)), 2)
        solution[f'Job{i}'] = f'M{machine_id}'

    # 解析C_max
    c_max = sum(int(bitstring[n * N + h]) * (2 ** h) for h in range(l))
    solution['C_max'] = c_max
    return solution


# 显示最优解
max_prob_state = max(counts, key=counts.get)
print("最优解比特串:", max_prob_state)
print("解码方案:", decode_solution(max_prob_state))